In [1]:
# for colab compatibility
!pip install shap
!pip install ppscore
!pip install xgboost
!pip install lightgbm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
%run data_load_wrapper.ipynb

In [3]:
from tqdm.notebook import trange
import sklearn.datasets as data
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn import *
from sklearn.feature_selection import *
import matplotlib as plt
import seaborn as sns
from boruta import BorutaPy
import shap
import numpy as np
import ppscore as pps
from datetime import datetime
import lightgbm as lgbm
import xgboost

In [4]:
# mse_test = metrics.matthews_corrcoef(y_test, res_test)
# mse_train = metrics.matthews_corrcoef(y_train, res_train)     

In [5]:
def sklearn_to_df(dataset, with_target=False):
    if with_target==False:
        df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
        y = dataset.target
        return df, y
    if with_target==True:
        df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
        y = dataset.target
        df = df.join(pd.Series(y, name='target'))
        return df

def get_feat_imp(imp_type, m, x_train, y_train, random_state):
    imp_type_dict = {
        'mutual_info' : 'mutual_info_classif(x_train, y_train)',
        'f_test' : 'f_classif(x_train, y_train)[0]',       
        'gini' : "m.feature_importances_",
        'pimp' : f"""permutation_importance(m, x_train, y_train, n_repeats=5, 
        random_state={random_state}).importances_mean""",
        'shap' : "abs(np.array(shap.TreeExplainer(m).shap_values(x_train))).sum(axis=0)",
        'pps' : "pps.predictors(x_train.join(y_train), 'target', sorted=False, output='df').model_score",
        'pearson' : "abs(x_train.join(y_train).corr(method='pearson').target).drop('target', axis=0)",
        'spearman': "abs(x_train.join(y_train).corr(method='spearman').target).drop('target', axis=0)",
        'kendall' : "abs(x_train.join(y_train).corr(method='kendall').target).drop('target', axis=0)"
    }
#     print(eval(imp_type_dict.get(imp_type)))
    return eval(imp_type_dict.get(imp_type))

def reduce_dataset(x_train, x_test, drop_index):
    x_train = x_train.drop(x_train.columns[drop_index], axis=1)
    x_test = x_test.drop(x_test.columns[drop_index], axis=1)
    return x_train, x_test

def eval_reduced(x_train, y_train, x_test, y_test, m, imp_type, 
                 dataset, dataset_str, random_state, dataset_state):
    reduce_stats = []
    columns_init = np.array(dataset.feature_names)
#     dataset_name = dataset.filename.split('/')[-1]
    for i in range(len(x_train.columns)):
        mse_train, mse_test, drop_index, feat = model_fit(x_train, y_train, 
                                                     x_test, y_test, m, imp_type,
                                                          random_state)
        global_index = np.where(columns_init==feat);
        global_index = global_index[0][0]
        reduce_stats.append((i, global_index, feat, 
                             mse_train, mse_test, 
                             # get class name, remove trash
                             str(m.__class__).split('.')[-1].replace("\'", '').strip('>'), 
                             imp_type, dataset_str, x_train.columns, 
                             random_state, dataset_state))
        x_train, x_test = reduce_dataset(x_train, x_test, drop_index)
    return reduce_stats

def model_fit(x_train, y_train, x_test, y_test, m, imp_type, random_state):
#     print(x_train.shape, y_train.shape)
#     print(x_test.shape, y_test.shape)
#     refactor1 into function get_metric(type={train, test})
    m.fit(x_train, y_train)
    res_train = m.predict(x_train)
    res_test = m.predict(x_test)
    mse_test = metrics.matthews_corrcoef(y_test, res_test)
    mse_train = metrics.matthews_corrcoef(y_train, res_train)     
    # get the index for feature with lowest global feature importance
    imp = get_feat_imp(imp_type, m, x_train, y_train, random_state)
    argmin_fi_index = imp.argmin()
    feat = x_train.columns[argmin_fi_index]
    return mse_train, mse_test, argmin_fi_index, feat

# def eval_reduced_one_shot(x_train, y_train, x_test, y_test, m, imp_type, 
#                  dataset, dataset_str, random_state, dataset_state):
    
#     reduce_stats = []
#     columns_init = np.array(x_train.columns)
#     m.set_params(random_state = random_state)
#     m.fit(x_train, y_train)
# #     print(x_train.shape)
# #     print(get_feat_imp(imp_type, m, x_train, y_train, random_state).shape)
#     model_str = str(m.__class__).split('.')[-1].replace("\'", '').strip('>')
# #     print(str(m.__class__).split('.')[-1].replace("\'", '').strip('>') == 'XGBClassifier')
#     if model_str != 'XGBClassifier':
#         fi_scores = get_feat_imp(imp_type, m, x_train, y_train, random_state)
#         print(f'not xgbclas {fi_scores.shape}')
# #         fi_scores = fi_scores.sum(axis=0)
#     elif model_str == 'XGBClassifier' and dataset_str == 'wine':      
#         fi_scores = get_feat_imp(imp_type, m, x_train, y_train, random_state)
#         fi_scores = fi_scores.sum(axis=0)
#     else:
#         fi_scores = get_feat_imp(imp_type, m, x_train, y_train, random_state)
#         fi_scores = fi_scores.sum(axis=0)
#         print(fi_scores.shape)
        
# #         print(f'imp: {imp_type} \n model: {m} \n x_shape: {x_train.shape} \n y_shape: {y_train.shape}')
# #         print(f'arr_scores: {fi_scores} \n shape: {fi_scores.shape}')
# #     print(fi_scores)
# #     print(fi_scores.shape)
# #     print(x_train.shape, y_train.shape)
#     for i in range(len(x_train.columns)):
#         mse_train, mse_test, drop_index, feat = model_fit_one_shot(x_train, y_train, 
#                                                      x_test, y_test, m, fi_scores, 
#                                                         random_state)
#         global_index = np.where(columns_init==feat);
#         global_index = global_index[0][0]
# #         print(global_index) 
# #         print(fi_scores)
#         reduce_stats.append((i, global_index, feat, 
#                              mse_train, mse_test, 
#                              # get class name, remove trash
#                              str(m.__class__).split('.')[-1].replace("\'", '').strip('>'), 
#                              imp_type, dataset_str, x_train.columns, 
#                              random_state, dataset_state))
#         x_train, x_test = reduce_dataset(x_train, x_test, drop_index)
# #         print(f'iter {i}')
# #         print(f'local_index: {drop_index}')
# #         print(f'global_index: {global_index}, \n fi before: \n {fi_scores}')
# #         print(fi_scores.shape)
#         fi_scores = np.array(fi_scores)
#         fi_scores = np.delete(fi_scores, drop_index)
# #         print(f'global_index: {global_index}, \n fi after: \n {fi_scores}')              
#     return reduce_stats

def model_fit_one_shot(x_train, y_train, x_test, y_test, m, fi_scores, 
                       random_state):
#     print(x_train.shape, y_train.shape)
#     print(fi_scores.shape)
#     refactor1 into function get_metric(type={train, test})
    m.set_params(random_state=random_state)
    m.fit(x_train, y_train)
    res_train = m.predict(x_train)
    res_test = m.predict(x_test)
    mse_test = metrics.matthews_corrcoef(y_test, res_test)
    mse_train = metrics.matthews_corrcoef(y_train, res_train)     
    # get the index for feature with lowest global feature importance
    imp = fi_scores
    argmin_fi_index = imp.argmin()
    
    feat = x_train.columns[argmin_fi_index]
    return mse_train, mse_test, argmin_fi_index, feat

In [6]:
def get_stats(imp_types, models, datasets, debug=False, random_states=[0], dataset_states=[0]):
    results = []
    datasets_str = list(datasets.keys())
    datasets_data = datasets.items()
    random_state = 0
    
    for ds in trange(len(dataset_states), desc='Data state progress'):
        dataset_state = dataset_states[ds]
        for r in trange(len(random_states), desc='Model random state progress'):
            random_state = random_states[r]

            for j in trange(len(datasets_str), desc=f'Dataset progress'):
                dataset_str = datasets_str[j]
                dataset = datasets.get(datasets_str[j])
                df = sklearn_to_df(dataset, with_target=True)
                x_train, x_test = model_selection.train_test_split(df, 
                                                                   test_size=0.3, 
                                                                   random_state=dataset_state,
                                                                   stratify=df.target
                                                                  )
                columns_init = x_train.columns
                y_train = pd.DataFrame(df.target[x_train.index], 
                                       index=x_train.index)
                y_test = pd.DataFrame(df.target[x_test.index], 
                                      index=x_test.index)
                # remove target column from x_train, x_test
                x_train = x_train.drop('target', axis=1)
                x_test = x_test.drop('target', axis=1)

                for k in trange(len(models), desc=f'Model progress for dataset {j}'):
                    m = models[k]
#                     print(x_train, y_train, x_test, y_test)
                    m.set_params(random_state=random_state)
                    m.fit(x_train, y_train.target)
                    res_test = m.predict(x_test)
                    res_train = m.predict(x_train)
                    mse_test = metrics.matthews_corrcoef(y_test, res_test)
                    mse_train = metrics.matthews_corrcoef(y_train, res_train)    

                    for i in trange(len(imp_types), desc=f'Importance type progress for model {k}'):
                        
                        if imp_types[i] in ['shap', 'pimp', 'pearson', 'spearman', 'kendall',
                                            'pps', 'mutual_info', 'f_test']:   
                            result = eval_reduced_one_shot(x_train, y_train, x_test, y_test, 
                                                           m, imp_types[i], 
                                                           dataset, dataset_str, 
                                                           random_state, dataset_state)
                        else:
                            result = eval_reduced(x_train, y_train.target, x_test, y_test, 
                                                  m, imp_types[i], dataset, dataset_str, 
                                                  random_state, dataset_state)
                        results.extend(result)
                df_res = pd.DataFrame(results, columns=['iteration', 'feat_ind', 'feat_name',
                                                    'mcc_train', 'mcc_test','model', 'imp_type', 
                                                    'dataset', 'features_lst', 
                                                    'random_state', 'data_split'])
        # shift features by 1 row, replace 0th iteration with 'full_set'
        df_res['dropped_feature'] = df_res['feat_name'].shift(1)
        df_res['dropped_feature'] = df_res['dropped_feature'].where(df_res['iteration'] != 0, 'full_set')
        if debug==True:
            repr_lst = ['iteration', 'dropped_feature', 'feat_ind', 'feat_name',
                        'mcc_train', 'mcc_test','model', 'imp_type', 
                        'dataset', 'features_lst','random_state', 'data_split']
        else:
            repr_lst = ['iteration','dropped_feature','mse_train',
         'mse_test','model','imp_type', 'dataset', 'random_state', 'data_split']
        df_res = df_res[repr_lst]
    return df_res

In [7]:
datasets  = {
    'wine' : data.load_wine(),
    'breast_cancer': data.load_breast_cancer(),    
    'phishing' : read_phishing(),
    'mushroom' : read_mushroom()
}
imp_types = [
             'gini',
             'mutual_info',
             'f_test',
             'pimp', 
             'shap', 
             'pearson', 
             'spearman', 
             'kendall', 
             'pps'
            ]
models = [
          tree.DecisionTreeClassifier(random_state=0), 
          ensemble.RandomForestClassifier(random_state=0, n_jobs=3), 
          xgboost.XGBClassifier(random_state=0, n_jobs=3, use_label_encoder=False,
                               verbosity=0),
         ]

random_states = [i for i in range(10)]

dataset_states = [i for i in range(10)]

In [8]:
# dataset_states = [i for i in range(8,10)]
# dataset_states

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
def eval_reduced_one_shot(x_train, y_train, x_test, y_test, m, imp_type, 
                 dataset, dataset_str, random_state, dataset_state):
    
    model_str = str(m.__class__).split('.')[-1].replace("\'", '').strip('>')
    reduce_stats = []
    columns_init = np.array(x_train.columns)
    m.set_params(random_state = random_state)
    print(m.__class__, x_train.shape, y_train.shape, dataset_str, imp_type)
    if  dataset_str == 'wine' and model_str == 'XGBClassifier':      
        m.set_params(num_class = 3)
    m.fit(x_train, y_train)
#     print(dataset_str, imp_type, m.__class__, x_train.shape, y_train.shape, random_state)
    if imp_type == 'shap' and model_str != 'XGBClassifier':
        fi_scores = get_feat_imp(imp_type, m, x_train, y_train, random_state)
#         print(1)
#         print(fi_scores.shape)
        fi_scores = fi_scores.sum(axis=0)
    elif model_str == 'XGBClassifier' and model_str == 'shap':
        fi_scores = get_feat_imp(imp_type, m, x_train, y_train, random_state)
#         fi_scores = fi_scores.sum(axis=0)
#         print(2)
#         print(fi_scores.shape)
    elif model_str == 'XGBClassifier' and dataset_str == 'wine' and imp_type == 'shap':
#         print(3)
        fi_scores = get_feat_imp(imp_type, m, x_train, y_train, random_state)
        fi_scores = fi_scores.sum(axis=0)
    elif model_str == 'XGBClassifier' and imp_type == 'shap':
#         print(4)
        fi_scores = get_feat_imp(imp_type, m, x_train, y_train, random_state)
        fi_scores = fi_scores.sum(axis=0)
    else:
#         print(5)
        fi_scores = get_feat_imp(imp_type, m, x_train, y_train, random_state)
        
    for i in range(len(x_train.columns)):
        mse_train, mse_test, drop_index, feat = model_fit_one_shot(x_train, y_train, 
                                                     x_test, y_test, m, fi_scores, 
                                                        random_state)
        global_index = np.where(columns_init==feat);
        global_index = global_index[0][0]
        reduce_stats.append((i, global_index, feat, 
                             mse_train, mse_test, 
                             # get class name, remove trash
                             str(m.__class__).split('.')[-1].replace("\'", '').strip('>'), 
                             imp_type, dataset_str, x_train.columns, 
                             random_state, dataset_state))
        x_train, x_test = reduce_dataset(x_train, x_test, drop_index)
#         print(f'iter {i}')
#         print(f'local_index: {drop_index}')
#         print(f'global_index: {global_index}, \n fi before: \n {fi_scores}')
#         print(fi_scores.shape)
        fi_scores = np.array(fi_scores)
        fi_scores = np.delete(fi_scores, drop_index)
#         print(f'global_index: {global_index}, \n fi after: \n {fi_scores}')              
    return reduce_stats

In [11]:
df_res = get_stats(imp_types, models, datasets, debug=True, 
                   random_states = random_states, dataset_states=dataset_states)

Data state progress:   0%|          | 0/2 [00:00<?, ?it/s]

Model random state progress:   0%|          | 0/10 [00:00<?, ?it/s]

Dataset progress:   0%|          | 0/4 [00:00<?, ?it/s]

Model progress for dataset 0:   0%|          | 0/3 [00:00<?, ?it/s]

Importance type progress for model 0:   0%|          | 0/9 [00:00<?, ?it/s]

...

<class 'sklearn.tree._classes.DecisionTreeClassifier'> (5686, 22) (5686, 1) mushroom mutual_info
<class 'sklearn.tree._classes.DecisionTreeClassifier'> (5686, 22) (5686, 1) mushroom f_test
<class 'sklearn.tree._classes.DecisionTreeClassifier'> (5686, 22) (5686, 1) mushroom pimp
<class 'sklearn.tree._classes.DecisionTreeClassifier'> (5686, 22) (5686, 1) mushroom shap
<class 'sklearn.tree._classes.DecisionTreeClassifier'> (5686, 22) (5686, 1) mushroom pearson
<class 'sklearn.tree._classes.DecisionTreeClassifier'> (5686, 22) (5686, 1) mushroom spearman
<class 'sklearn.tree._classes.DecisionTreeClassifier'> (5686, 22) (5686, 1) mushroom kendall
<class 'sklearn.tree._classes.DecisionTreeClassifier'> (5686, 22) (5686, 1) mushroom pps


Importance type progress for model 1:   0%|          | 0/9 [00:00<?, ?it/s]

<class 'sklearn.ensemble._forest.RandomForestClassifier'> (5686, 22) (5686, 1) mushroom mutual_info
<class 'sklearn.ensemble._forest.RandomForestClassifier'> (5686, 22) (5686, 1) mushroom f_test
<class 'sklearn.ensemble._forest.RandomForestClassifier'> (5686, 22) (5686, 1) mushroom pimp
<class 'sklearn.ensemble._forest.RandomForestClassifier'> (5686, 22) (5686, 1) mushroom shap
<class 'sklearn.ensemble._forest.RandomForestClassifier'> (5686, 22) (5686, 1) mushroom pearson
<class 'sklearn.ensemble._forest.RandomForestClassifier'> (5686, 22) (5686, 1) mushroom spearman
<class 'sklearn.ensemble._forest.RandomForestClassifier'> (5686, 22) (5686, 1) mushroom kendall
<class 'sklearn.ensemble._forest.RandomForestClassifier'> (5686, 22) (5686, 1) mushroom pps


Importance type progress for model 2:   0%|          | 0/9 [00:00<?, ?it/s]

<class 'xgboost.sklearn.XGBClassifier'> (5686, 22) (5686, 1) mushroom mutual_info
<class 'xgboost.sklearn.XGBClassifier'> (5686, 22) (5686, 1) mushroom f_test
<class 'xgboost.sklearn.XGBClassifier'> (5686, 22) (5686, 1) mushroom pimp
<class 'xgboost.sklearn.XGBClassifier'> (5686, 22) (5686, 1) mushroom shap
<class 'xgboost.sklearn.XGBClassifier'> (5686, 22) (5686, 1) mushroom pearson
<class 'xgboost.sklearn.XGBClassifier'> (5686, 22) (5686, 1) mushroom spearman
<class 'xgboost.sklearn.XGBClassifier'> (5686, 22) (5686, 1) mushroom kendall
<class 'xgboost.sklearn.XGBClassifier'> (5686, 22) (5686, 1) mushroom pps


In [12]:
# # debug
# a = data.load_breast_cancer()
# m = xgboost.XGBClassifier(random_state=0, n_jobs=-1, use_label_encoder=False,
#                                verbosity=0)
# split_arr = model_selection.train_test_split(a.data, a.target, test_size=0.3, random_state=0)
# x_train = split_arr[0] 
# y_train = split_arr[2]
# m.fit(x_train, y_train)
# print(x_train.shape)
# wat = abs(np.array(shap.TreeExplainer(m).shap_values(x_train))).sum(axis=0)
# print(wat.shape)

In [13]:
# import seaborn as sns

# plot = sns.lineplot(x='iteration', y='mse_test', hue='imp_type',
#             data=df_res.query('model == "RandomForestClassifier" and dataset == "breast_cancer"')
#             )

In [14]:
# import seaborn as sns

# plot = sns.lineplot(x='iteration', y='mse_test', hue='imp_type',
#             data=df_res.query('dataset == "wine" and model == "RandomForestClassifier"')
#             )

In [15]:
# export results to local dir as csv file
df_res.to_csv(f'classification_real_data_{datetime.today().strftime("%Y-%b-%d-%H:%M:%S")}.csv',
             index=False)